In [1]:
import plotly.express as px
import re
import plotly.io as pio
# pio.renderers.default = "browser"
pio.renderers.default = "notebook_connected"
from collections import OrderedDict
import argparse
from scipy.stats import gmean
import numpy as np
import plotly.graph_objects as go
import plotly.figure_factory as ff
import os
from util import bench_num_name_map, bench_num_unit_map, no_loop_end_list, bench_label_map

In [2]:
# def construct_argparser():
#     parser = argparse.ArgumentParser(description='plot')
#     parser.add_argument('-t',
#                         '--tag',
#                         help='tag',
#                         default='base_refrate_orig_config-m64.0000',
#                         )
    
#     parser.add_argument('-n',
#                         '--num',
#                         nargs='+',
#                         help='bench num',
#                         default=[505]
#                         )
#     return parser

In [3]:
# parser = construct_argparser()
# args = parser.parse_args()

# specify spec number below

In [4]:
# bench_num=args.num[0]
# 5503, 505, 507, 525, 557
bench_num=525
bench_name=bench_num_name_map[bench_num]
bench_unit = bench_num_unit_map[bench_num]

In [5]:
# tag=args.tag
tag=f'base_refrate_orig_{bench_label_map[bench_num]}-m64.0000'
# tag='base_refrate_orig_config-m64.0000'

In [6]:
file=f"/home/zhewen/repo/cpu2017/benchspec/CPU/{bench_num}.{bench_name}/run/run_{tag}/out.txt"

In [7]:
find_comma_num = lambda line: int(re.findall(r'\b\d[\d,.]*\b', line)[1].replace(',',''))

In [8]:
file1 = open(file, 'r')
Lines = file1.readlines()
  
count = 0
cycles_core = []
cycles_atom = []
insns_core = []
insns_atom = []
begins = []
ends = []
# Strips the newline character
for line in Lines:
    count += 1
    if 'cpu_core/cycles/' in line: 
        if '<not counted>' in line: 
            cycles_core.append(0)
            continue
        cycle_core=find_comma_num(line)
        cycles_core.append(cycle_core)
    elif 'cpu_atom/cycles/' in line:
        if 'not counted' in line: 
            cycles_atom.append(0)
            continue
        cycle_atom=find_comma_num(line)
        cycles_atom.append(cycle_atom)
    elif 'cpu_core/instructions/' in line:
        if 'not counted' in line: 
            insns_core.append(0)
            continue
        insn_core=find_comma_num(line)
        insns_core.append(insn_core)
    elif 'cpu_atom/instructions/' in line:
        if 'not counted' in line: 
            insns_atom.append(0)
            continue
        insn_atom=find_comma_num(line)
        insns_atom.append(insn_atom)
    elif 'loop ' in line or 'loop_begin' in line:
        begin=find_comma_num(line)
        begins.append(begin)
    elif 'loop_end' in line:
        end=find_comma_num(line)
        ends.append(end)
        

In [9]:
cycles = [core+atom for core,atom in zip(cycles_core, cycles_atom)]
insns = [core+atom for core,atom in zip(insns_core, insns_atom)]

In [10]:
if ends and all(elem == 0 or elem == 1 for elem in ends):
    print('end valid')
else: assert False, 'end contains other'
if begins and all(elem == 0 or elem == 1 for elem in begins):
    print('begin valid')
else: assert False, 'begin contains other'


end valid
begin valid


In [11]:
if bench_num not in no_loop_end_list: assert len(ends) != 0

In [12]:
cpis = [c/i for c,i in zip(cycles,insns)]

In [13]:
index_begin=[ i for i in range(len(begins)) if begins[i] == 1 ]
if bench_num in no_loop_end_list: 
    index_begin = index_begin[0:-1]#ignore last begin
index_end =[ i for i in range(len(ends)) if ends[i] == 1 ]
# if bench_num not in no_loop_end_list: 
assert len(index_begin)==len(index_end),print(len(index_begin),len(index_end))

In [14]:
ticks=[]
i = 0 
for b, e in zip(index_begin,index_end):
    tick=e-b
    if i!= 0: ticks.append(tick)
    i += 1
avg_ticks=float(sum(ticks)/len(ticks))
avg_ms = avg_ticks*float(bench_unit)
print(f'{avg_ms} ms, total {len(index_begin)} iterations')

2444.4444444444443 ms, total 10 iterations


In [15]:
x=np.arange(len(cpis))
fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=cpis,
                mode='lines',
                name=f'cpi',
                ))
# fig.update_yaxes(type="log")
fig.update_layout(title=f'{bench_name} phase analysis, per iteration latency={avg_ms} ms',
                   xaxis_title=f'nx{bench_unit} ms',
                   yaxis_title='cpi')

if bench_num not in no_loop_end_list:
    for b,n in zip(index_begin, index_end):
        index = index_begin.index(b)
        if (index % 2) == 0: color = 'red' 
        else: color = "green"
        fig.add_vrect(x0=b, x1=n, line_width=0, fillcolor=color, opacity=0.1)
else:
    for i in range(len(index_begin)-1):
        if (i % 2) == 0: color = 'red'
        else: color = "green"
        fig.add_vrect(x0=index_begin[i], x1=index_begin[i+1], line_width=0, fillcolor=color, opacity=0.1)
fig.show()
    